In [1]:
import pandas as pd

df = pd.read_csv("portfolio_articles_3.csv", sep="|", dtype='str')

In [4]:
df

,url,title,content,date,error
0,https://www.portfolio.hu/gazdasag/20200616/666...,666 új koronavírusos fertőzött egy nap alatt U...,Továbbra sem lassul Ukrajnában a koronavírus-f...,2020/06/16,NaN
1,https://www.portfolio.hu/gazdasag/20200616/kor...,"Koronavírus: megvan, mikor érhet véget a rendk...","A kormány üdvözli az Országgyűlés döntését , a...",2020/06/16,NaN
2,https://www.portfolio.hu/gazdasag/20200616/kor...,Koronavírus: megszűnik a veszélyhelyzet Magyar...,Elfogadta az Országgyűlés a koronavírus-járván...,2020/06/16,NaN
3,https://www.portfolio.hu/gazdasag/20200616/hat...,Hatalmas visszaesés vár Svájcra,A svájci bruttó hazai terméket több mint 100 m...,2020/06/16,NaN
4,https://www.portfolio.hu/gazdasag/20200616/kul...,Külföldi tőkebefektetők segítségével válna ors...,"Még márciusban, a koronavírus hazai megjelenés...",2020/06/16,NaN
...,...,...,...,...,...
15388,https://www.portfolio.hu/gazdasag/20180123/pen...,Pénzt kapnak a görögök,Jóváhagyták a Görögországot segítő nemzetközi ...,2018/01/23,NaN
15389,https://www.portfolio.hu/gazdasag/20180123/bel...,Beleszállt a Facebookba és a Google-ba a média...,Rupert Murdoch amerikai médiamágnás nyílt levé...,2018/01/23,NaN
15390,https://www.portfolio.hu/gazdasag/20180123/veg...,"Véget ért a kormányzati leállás az USA-ban, de...",A Demokrata Párt hétfőn megállapodott a republ...,2018/01/23,NaN
15391,https://www.portfolio.hu/gazdasag/20180122/jov...,Jóváhagyták a görög mentőcsomag következő rész...,"Jóváhagyták a görög mentőcsomag következő, 6,7...",2018/01/22,NaN


In [3]:
df = df[
    (df["title"] != "Title not found") & 
    ~df["content"].fillna("").str.contains("Content not found") & 
    ~df["content"].fillna("").str.strip().eq("") & 
    ~df["content"].fillna("").str.contains("Error fetching content") & 
    df["error"].isnull()
]

In [5]:
df = df.drop("error", axis=1)
df['full_text'] = df['title'] + ". " + df['content']
df = df.reset_index(drop=True)

In [6]:
df.to_csv("portfolio_articles_3.csv", sep="|", encoding='utf-8', index=False)

In [8]:
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
from google import genai
from google.genai import types

client = genai.Client(api_key="")

# Function to translate each text
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main processing loop
async def main(df):
    try:
        df_translated = pd.read_csv("portfolio_articles_3_translated.csv", sep="|", encoding='utf-8')
        processed_ids = set(df_translated.index)
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
        processed_ids = set()

    batch_size = 75  # Number of rows to process per batch
    start_index = max(processed_ids) + 1 if processed_ids else 0

    for i in range(start_index, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        batch = await translate_batch_async(batch)
        
        # Append to the result DataFrame
        df_translated = pd.concat([df_translated, batch], ignore_index=True)
        
        # Save to CSV after each batch
        df_translated.to_csv("portfolio_articles_3_translated.csv", index=False, sep="|", encoding='utf-8')
        
        print(f"Batch {i // batch_size + 1} processed and saved. Total articles translated: {len(df_translated)}")


# Run the asynchronous main function
await main(df)

C:\Users\tothk\AppData\Local\Temp\ipykernel_2868\2871198858.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 1 processed and saved. Total articles translated: 75


C:\Users\tothk\AppData\Local\Temp\ipykernel_2868\2871198858.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 2 processed and saved. Total articles translated: 150


C:\Users\tothk\AppData\Local\Temp\ipykernel_2868\2871198858.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


...

Batch 201 processed and saved. Total articles translated: 15075


C:\Users\tothk\AppData\Local\Temp\ipykernel_2868\2871198858.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 202 processed and saved. Total articles translated: 15150


C:\Users\tothk\AppData\Local\Temp\ipykernel_2868\2871198858.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 203 processed and saved. Total articles translated: 15159


In [23]:
df_proba = pd.read_csv("portfolio_articles_3_translated.csv", sep="|", dtype='str')

In [13]:
df_proba[df_proba["translated_content"].isnull()].count()

full_text             48
translated_content     0
url                   48
title                 48
content               48
date                  48
dtype: int64

In [24]:
df_proba

,full_text,translated_content,url,title,content,date
0,666 új koronavírusos fertőzött egy nap alatt U...,Ukraine reports 666 new coronavirus cases in o...,https://www.portfolio.hu/gazdasag/20200616/666...,666 új koronavírusos fertőzött egy nap alatt U...,Továbbra sem lassul Ukrajnában a koronavírus-f...,2020/06/16
1,"Koronavírus: megvan, mikor érhet véget a rendk...",Coronavirus: It's known when the special legal...,https://www.portfolio.hu/gazdasag/20200616/kor...,"Koronavírus: megvan, mikor érhet véget a rendk...","A kormány üdvözli az Országgyűlés döntését , a...",2020/06/16
2,Koronavírus: megszűnik a veszélyhelyzet Magyar...,Coronavirus: State of Danger Ends in Hungary.\...,https://www.portfolio.hu/gazdasag/20200616/kor...,Koronavírus: megszűnik a veszélyhelyzet Magyar...,Elfogadta az Országgyűlés a koronavírus-járván...,2020/06/16
3,Hatalmas visszaesés vár Svájcra. A svájci brut...,Switzerland is expected to face a huge decline...,https://www.portfolio.hu/gazdasag/20200616/hat...,Hatalmas visszaesés vár Svájcra,A svájci bruttó hazai terméket több mint 100 m...,2020/06/16
4,Külföldi tőkebefektetők segítségével válna ors...,One of the largest Hungarian private healthcar...,https://www.portfolio.hu/gazdasag/20200616/kul...,Külföldi tőkebefektetők segítségével válna ors...,"Még márciusban, a koronavírus hazai megjelenés...",2020/06/16
...,...,...,...,...,...,...
15136,Pénzt kapnak a görögök. Jóváhagyták a Görögors...,The Greeks are getting money. The finance mini...,https://www.portfolio.hu/gazdasag/20180123/pen...,Pénzt kapnak a görögök,Jóváhagyták a Görögországot segítő nemzetközi ...,2018/01/23
15137,Beleszállt a Facebookba és a Google-ba a média...,Media mogul attacks Facebook and Google. Ameri...,https://www.portfolio.hu/gazdasag/20180123/bel...,Beleszállt a Facebookba és a Google-ba a média...,Rupert Murdoch amerikai médiamágnás nyílt levé...,2018/01/23
15138,"Véget ért a kormányzati leállás az USA-ban, de...","The US government shutdown has ended, but it c...",https://www.portfolio.hu/gazdasag/20180123/veg...,"Véget ért a kormányzati leállás az USA-ban, de...",A Demokrata Párt hétfőn megállapodott a republ...,2018/01/23
15139,Jóváhagyták a görög mentőcsomag következő rész...,The next installment of the Greek bailout pack...,https://www.portfolio.hu/gazdasag/20180122/jov...,Jóváhagyták a görög mentőcsomag következő rész...,"Jóváhagyták a görög mentőcsomag következő, 6,7...",2018/01/22


In [17]:
# Function to translate each text 
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main function to process and retry NaN translations
async def main():
    try:
        df_translated = pd.read_csv("portfolio_articles_3_translated.csv", sep="|", encoding='utf-8')
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
    
    # First pass: Process untranslated rows
    untranslated_rows = df_translated["translated_content"].isna().sum()
    print(f"Untranslated rows found: {untranslated_rows}")

    if untranslated_rows > 0:
        to_retry = df_translated[df_translated["translated_content"].isna()]
        df_translated.loc[to_retry.index, "translated_content"] = (await translate_batch_async(to_retry))["translated_content"].values

        # Save the corrected translations
        df_translated.to_csv("portfolio_articles_3_translated.csv", index=False, sep="|", encoding='utf-8')
        print(f"Retry complete! Remaining untranslated rows: {df_translated['translated_content'].isna().sum()}")
    
    print("Translation process complete!")

# Run the asynchronous main function
await main()


Untranslated rows found: 21


C:\Users\tothk\AppData\Local\Temp\ipykernel_2868\2103462649.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Retry complete! Remaining untranslated rows: 18
Translation process complete!


In [20]:
df_proba = pd.read_csv("portfolio_articles_3_translated.csv", sep="|", dtype='str')

df_proba.dropna(subset=['translated_content'], inplace=True)
df_proba.reset_index(drop=True, inplace=True)

df_proba.to_csv("portfolio_articles_3_translated.csv", index=False, sep="|", encoding='utf-8')

In [22]:
df_proba.loc[1234,"translated_content"]

'Another European airline could be saved from collapse by the state.\n\nThe Czech state would buy 100% of Smartwings, the parent company of Czech Airlines, or provide state-guaranteed loans to the company, which is in a difficult situation due to the coronavirus, Czech Industry Minister Karel Havlicek announced, according to a Reuters report.\n\nThe Minister told Czech state television that they are in negotiations with Smartwings and expect to reach an agreement by the end of June. He added that the company is strategically important. According to Reuters, Finance Minister Alena Schillerova supports the idea of \u200b\u200bbuying the aviation group.\n\nSmartwings added that it welcomes the negotiations. The company is grounding its aircraft due to travel restrictions introduced to curb the coronavirus epidemic. One of its subsidiaries, Czech Airlines, will restart some flights on Monday.\n\nIn 2019, the group carried 8.2 million passengers, employs 2,500 people, and is the largest cus